## Big Data Project on Predicting Taxi Fare Price in city of Chicago using Linear Regression

Getting the Data
Get the data from
https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=chicago_taxi_trips&page=dataset&project=big-data-project-396823&supportedpurview=project&ws=!1m9!1m4!4m3!1sbigquery-public-data!2schicago_taxi_trips!3staxi_trips!1m3!3m2!1sbigquery-public-data!2schicago_taxi_trips

The data has the following
1.   Total logical bytes: 75.75 GB
2.   Number of rows: 208,943,621

For the sake of this project lets extract only 5000 rows from the data

# Use Datalake on AWS S3 to store data for the following reasons

## Schema Evolution:
Data lakes enable schema-on-read, meaning that you can apply structure to the data during analysis rather than enforcing a fixed schema on ingest. This flexibility is beneficial when dealing with evolving data sources and schema changes over time.

## Cost-Efficiency for Storage:
Data lakes, like Amazon S3, offer cost-effective storage options for large volumes of data. Since the Chicago taxi fare data might grow over time, you can leverage a pay-as-you-go pricing model, storing the data without incurring significant costs.

## Handling High Volume and Velocity:
If you're dealing with large volumes of data or high data velocity (frequent updates), data lakes can handle the scale more effectively. They're designed to handle big data scenarios and can accommodate rapid growth.

## Scalability and Future-Proofing:
Data lakes offer high scalability and can adapt to future data needs. As new data sources emerge and analytical requirements evolve, a data lake can provide a more scalable and adaptable solution.

##install pyspark

In [4]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=a89313ac234b1e342e1855afdcda1bf6a99b34c7851b2bedf44ac2239bcf4c05
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


## import all the required libraries

In [72]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

#Download the data from S3

In [73]:
import requests

url = "https://fractalcards-dev.s3.amazonaws.com/Chicago_taxi_fare_5000.csv"
response = requests.get(url)

if response.status_code == 200:
    with open("Chicago_taxi_fare_5000.csv", "wb") as file:
        file.write(response.content)
    print("CSV file downloaded successfully.")
else:
    print("Failed to download the CSV file.")


CSV file downloaded successfully.


start spark with the app name PricePrediction

In [74]:
spark = SparkSession.builder.appName("PricePrediction").getOrCreate()


Load data

In [75]:
csv_file_path = "Chicago_taxi_fare_5000.csv"

# Read data from the downloaded CSV file
taxi_data = spark.read.csv("/content/"+csv_file_path, header=True, inferSchema=True)


In [76]:
# Clean the data and handle inconsistencies
cleaned_data = taxi_data.filter(
    col("trip_seconds").isNotNull() &
    col("trip_miles").isNotNull() &
    col("pickup_community_area").isNotNull() &
    col("fare").isNotNull()
)

# Feature columns and assembler
feature_columns = ["trip_seconds", "trip_miles", "pickup_community_area"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_data = assembler.transform(cleaned_data)

# Split data into training and testing sets
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=123)

# Train a linear regression model
lr = LinearRegression(featuresCol="features", labelCol="fare")
lr_model = lr.fit(train_data)


In [77]:
# Evaluate the model
test_results = lr_model.evaluate(test_data)
print("Root Mean Squared Error (RMSE):", test_results.rootMeanSquaredError)
print("R2:", test_results.r2)

# Make predictions
predictions = lr_model.transform(test_data)
predictions.select("fare", "prediction").show()

# Stop the Spark session
spark.stop()


Root Mean Squared Error (RMSE): 13.325519339163872
R2: 0.32574266647997696
+----+-----------------+
|fare|       prediction|
+----+-----------------+
|3.25|8.742912018173813|
|3.25|  8.8407249708915|
|3.25|8.873329288464063|
|12.0|8.873329288464063|
|3.25|8.905933606036626|
|3.25|8.905933606036626|
|3.25|8.905933606036626|
|3.25|8.905933606036626|
|3.25|8.905933606036626|
|3.25|8.905933606036626|
| 3.5|8.905933606036626|
|3.25| 8.97114224118175|
|80.0| 8.97114224118175|
|3.25|9.036350876326877|
|3.25|   9.101559511472|
|3.25|9.134163829044564|
|3.25|9.166768146617125|
|3.25|9.166768146617125|
|3.25|9.166768146617125|
|3.25|9.297185416907375|
+----+-----------------+
only showing top 20 rows



In [ ]:
We executed our Spark application from the command line interface, initiating a job that performed data analysis and prediction using the Linear Regression model. As the job ran, we monitored its progress and outcome through Amazon Web Services (AWS) tools.